In [156]:
# Get yer imports here
import pandas as pd
import numpy as np
import math
import scipy
from scipy.io import loadmat
from scipy.stats import f_oneway
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

# I came back up here and wrote this later just because I wanted to have shorter logs to sort through
DEBUG_MODE = False

def debug_print(str_to_print):
    if DEBUG_MODE:
        print(str_to_print)

In [10]:
# GRAVE OF ATTEMPT ONE
#file_name = 'EEG data and psychometric results from children with learning difficulties/baseline/Sub0001_B.gdf'
#test = mne.io.read_raw_gdf(file_name)

# mne.io.read_raw_gdf(input_fname, eog=None, misc=None, stim_channel='auto', exclude=(), include=None, preload=False, verbose=None)

#print(test.info)
#test.to_data_frame()

In [45]:
# -------------------------------------------------------------------------------------------------------------
# THIS CELL IS ABOUT LOADING IN A TEST DATA AND JUST PRINTING IT.
# -------------------------------------------------------------------------------------------------------------

# grabbing opening mat files in python from this website. because i don't know how it works at all
# https://www.askpython.com/python/examples/mat-files-in-python
# ...aaaaaand this stopped being useful after like. the first step. turning this thing from a .mat file to
# some manner of data frame was all i really needed though, i can tool around with it from there until i
# get something workable.


annots = loadmat('hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180129_1818_G.mat')

#print(annots)

the_important_stuff = annots['data']
#print(the_important_stuff)

print(pd.DataFrame(the_important_stuff))



"""
item_list = list(annots)
for item in item_list:
    print("WOWEE THIS IS SOMETHING PRETTY IMPORTANT")
    print(item, annots[item])
    print("----------------------------------------")
"""


#print(type(annots))
#print(next(iter(annots)))

#print(annots[1])

#con_list = [[element for element in upperElement] for upperElement in annots['data']]
#print(con_list)

"""
con_list = [[element for element in upperElement] for upperElement in annots['obj_contour']]
print(con_list)
"""

pd.DataFrame(the_important_stuff)


                                                   0  \
0  [[[[[(array([[32.80834841]]), array([[314.1391...   

                                                   1  
0  [[[[[1480]], [[3720]], [[1]], [[800]], [[9]], ...  


,0,1
0,"[[[[[(array([[32.80834841]]), array([[314.1391...","[[[[[1480]], [[3720]], [[1]], [[800]], [[9]], ..."


In [3]:
# -------------------------------------------------------------------------------------------------------------
# THIS CELL IS ABOUT ATTEMPTING TO FURTHER EXTRACT LITERALLY ANYTHING FROM THIS TEST DATA.
# -------------------------------------------------------------------------------------------------------------

annots = loadmat('hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180129_1818_G.mat')

#print(annots)

the_important_stuff = annots['data']
#print(the_important_stuff)

#print(pd.DataFrame(the_important_stuff))

# UNREADABLE AS NUMPY FRAME. TO INVESTIGATE FURTHER I WILL NEED TO USE PD.DATAFRAME().

print(pd.DataFrame(the_important_stuff))

print("---------- cool divider line ---------")

#print("THIS STUFF IS THE DATA[1] THE DOC IS TALKING ABOUT")
data_1 = the_important_stuff[:,0]
#print(data_1)

#print("THIS STUFF IS THE DATA[2] THE DOC IS TALKING ABOUT"
data_2 = the_important_stuff[:,1]
#print(data_2)




df_1 = pd.DataFrame(data_1[0][0])
df_1.drop(labels=["RewardSchedule", "NmrPulsesConfig", "NmrPulses", "Key2", "Key3"], inplace=True, axis=1)



#print(pd.DataFrame(data_1[0][0])["LeftStimulusPosition"])

#pd.DataFrame(data_1[0][0][0][0][0])
# pd.DataFrame(data_1[0][0])

#df_1
# This was just to get a better look at the other half of the data
df_1.drop(labels=["LeftStimulusPosition", "RightStimulusPosition", "EyeWindows", "PhotoEventsConfig", "PhotoEvents", "StimEventsConfig", "StimEvents", "Key0", "Key1"], inplace=False, axis=1)




                                                   0  \
0  [[[[[(array([[32.80834841]]), array([[314.1391...   

                                                   1  
0  [[[[[1480]], [[3720]], [[1]], [[800]], [[9]], ...  
---------- cool divider line ---------


,dataStart,daqSampleRate,analogInId,digitalInId,digitalOutId,sourceDir,spikes,presenter,stimuliPerPhotoEvent,analogSampleRate,analog,digitalSampleRate,digital,photoEventsRhd,photoEventsRhdMSec
0,[[770]],[[10000]],"[[[Eye X], [Eye Y], [Pupil], [Photocell], [Sti...","[[[Key0], [Key1], [Key2], [Key3]]]","[[[Reward], [StimulationTrigger], [SpikeAccept...",[/data/luna/luna_l/GBW-0002_Neuro_fMRI/0001_SE...,"[[828.4], [922.0], [954.3], [984.7], [1040.0],...","[[[[[(array([[113]], dtype=uint8), array(['fix...","[[[[array(['doublePhoto'], dtype='<U11') array...","[[1000, 1000, 1000, 1000, 1000, 1000]]","[[-0.2835116982460022, -0.2764776349067688, -0...",[[10000]],"[[0], [0], [0], [0], [0], [0], [0], [0], [0], ...","[[7642.5, 8139.5, 14312.5, 18955.5, 19956.5, 2...","[[764.15, 813.85, 1431.15, 1895.45, 1995.55, 2..."


In [4]:
# -------------------------------------------------------------------------------------------------------------
# THIS CELL IS ABOUT GLUING MULTIPLE SESSIONS TOGETHER TO CREATE AN ACTUAL USEFUL(ish) TABLE.
# I shouldn't need to rerun this very often, thanfully. I just need to merge it all, and explore.
# -------------------------------------------------------------------------------------------------------------

full_data = None

filepath_list = ['hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180129_1818_G.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180130_1540_G.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180214_1740_B.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180215_1647_D.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180216_1346_H.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180226_1444_A.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180305_1526_D.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180305_1607_E.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180305_1704_F.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180305_1749_G.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180306_1444_C.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180306_1606_A.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180306_1743_B.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180307_1425_A.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180307_1558_B.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180308_1539_A.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180308_1715_B.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180417_1822_B.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180418_1440_A.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180418_1653_B.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180423_1744_D.mat',
                 'hbp-d000034_AttentionNetworksInMonkeys_pub/behavior/sub-xx_20180423_1911_E.mat'
                 ]

for filepath in filepath_list:
    debug_print("Reading file: " + filepath)
    file = loadmat(filepath)
    the_important_stuff = file['data']
    
    debug_print("Splitting file...")
    data_1 = the_important_stuff[:,0]
    data_2 = the_important_stuff[:,1]
    
    debug_print("Converting file in data...")
    df_1 = pd.DataFrame(data_1[0][0])
    df_1.drop(labels=["RewardSchedule", "NmrPulsesConfig", "NmrPulses", "PhotoEventsConfig", "StimEventsConfig", "Key2", "Key3", "daqSampleRate", "analogInId", "digitalInId", "digitalOutId", "sourceDir", "presenter", "stimuliPerPhotoEvent", "analogSampleRate", "digitalSampleRate"], inplace=True, axis=1)
    # Only some of the tables have this analogOutID column, and either way, it's entirely useless. It does make me have to put this single drop statement in its own separate line, which is annoying.
    # If I had more time, I'd try to coax the "presenter" and "stimuli per photo event" variables out of their horrendous formatting, but it's not worth it. 
    if "analogOutId" in df_1:
        df_1.drop(labels=["analogOutId"], inplace=True, axis=1)
    
    debug_print("Converting other file into data...")
    df_2 = pd.DataFrame(data_2[0][0])
    df_2.drop(labels=["Stimulation", "StimulationPower", "NmrPulses", "StimEvents", "Key2", "Key3", "Event", "Histogram", "PhotoEvents", "Key0", "Key1", "spikes"], inplace=True, axis=1)
    # Event and Histogram might have important data in them, but I don't know how to process them.
    # PhotoEvents, Key0, Key1, and spikes are dupe columns
    
    df_full = pd.concat([df_1, df_2], axis=1)
    
    debug_print("Merging data...")
    full_data = pd.concat([full_data, df_full], ignore_index=True, axis=0)
    
print("Done!")
full_data



Done!


,LeftStimulusPosition,RightStimulusPosition,EyeWindows,PhotoEvents,StimEvents,Key0,Key1,dataStart,spikes,analog,...,photoEventsRhd,photoEventsRhdMSec,Start,Stop,Answer,InterTrialInterval,Condition,ResponseTrial,TrialTiming,PhotoEventsRHD
0,"[[[[[32.80834841]], [[314.13915409]]]]]","[[[[[32.80834841]], [[134.13915409]]]]]",[[[[[-0.214634 ]\n [-0.536585 ]\n [ 0.17561 ]...,"[[1545.7, 1595.5, 2212.8, 2677.1, 2777.1, 3711...","[[176803.3, 177052.9, 182542.4, 182792.0, 1883...","[[91575.8, 258175.6, 386316.3, 388639.5, 53569...","[[12113.9, 17002.0, 21667.3, 27552.4, 56411.8,...",[[770]],"[[828.4], [922.0], [954.3], [984.7], [1040.0],...","[[-0.2835116982460022, -0.2764776349067688, -0...",...,"[[7642.5, 8139.5, 14312.5, 18955.5, 19956.5, 2...","[[764.15, 813.85, 1431.15, 1895.45, 1995.55, 2...",[[1480]],[[3720]],[[1]],[[800]],[[9]],[[0]],[[[[467.13333333 100.1 934.26666667]]]],"[[764.15, 813.85, 1431.15, 1895.45, 1995.55, 2..."
1,"[[[[[13.46757377]], [[308.20541614]]]]]","[[[[[13.46757377]], [[128.20541614]]]]]",[[[[[ 0.0882702]\n [-1.40488 ]\n [ 0.478514 ]...,"[[10239.8, 10288.6, 10890.4, 11390.9, 11591.1,...","[[15796.2, 16046.5, 24705.2, 24955.5, 27758.2,...","[[289559.0, 466222.6, 467862.5, 470716.4, 1750...","[[11864.1, 19415.9, 31670.0, 37606.1, 96241.3,...",[[9670]],"[[9675.3], [9796.1], [10073.1], [10075.7], [10...","[[0.5390381813049316, 0.5391367673873901, 0.53...",...,"[[5653.5, 6141.5, 12159.5, 17165.5, 19167.5, 1...","[[565.25, 614.05, 1215.85, 1716.45, 1916.65, 1...",[[10180]],[[11390]],[[4]],[[667]],[[0]],[[1]],[[[[500.5 100.1 900.9]]]],"[[565.25, 614.05, 1215.85]]"
2,"[[[[[27.21131657]], [[31.50426672]]]]]","[[[[[27.21131657]], [[-148.49573328]]]]]",[[[[[-0.145528]\n [-0.880778]\n [ 0.247967]\n ...,"[[1184.6, 1234.3, 1852.0, 2185.2, 2285.3, 3353...",[],"[[266555.2, 461743.7, 463431.6, 463844.6, 1078...","[[33017.1, 49572.9, 69973.1, 71299.2, 85777.0,...",[[770]],"[[774.3], [1051.6], [1057.8], [1107.5], [1110....","[[-1.309370756149292, -1.310192584991455, -1.3...",...,"[[4107.5, 4604.5, 10781.5, 14114.5, 15114.5, 2...","[[410.65, 460.35, 1078.05, 1411.35, 1511.35, 2...",[[1120]],[[3360]],[[1]],[[850]],[[10]],[[0]],[[[[ 333.66666667 100.1 1067.73333333]]]],"[[410.65, 460.35, 1078.05, 1411.35, 1511.35, 2..."
3,"[[[[[16.14765063]], [[51.77837553]]]]]","[[[[[16.14765063]], [[-128.22162447]]]]]",[[[[[-0.127235 ]\n [-0.997852 ]\n [ 0.26626 ]...,"[[6017.7, 6066.6, 6701.8, 6750.7, 7369.1, 7969...","[[13208.9, 13459.7, 24970.8, 25221.6, 36982.9,...","[[6466.6, 495471.5, 532818.9, 1073571.9, 13918...","[[14319.7, 29082.0, 40853.1, 49555.5, 58301.3,...",[[4260]],"[[6561.5], [6711.2], [6718.2], [6773.8], [6778...","[[0.07552260905504227, 0.0755554735660553, 0.0...",...,"[[17547.5, 18036.5, 24387.5, 24876.5, 31061.5,...","[[1754.65, 1803.55, 2438.65, 2487.55, 3106.05,...",[[5960]],[[6470]],[[3]],[[650]],[[6]],[[0]],[[[[ 367.03333333 100.1 1034.36666667]]]],"[[1754.65, 1803.55]]"
4,"[[[[[41.19192024]], [[334.57146374]]]]]","[[[[[41.19192024]], [[154.57146374]]]]]",[[[[[-0.0491876]\n [-0.965332 ]\n [ 0.268293 ]...,"[[1464.5, 1514.2, 2128.6, 2745.9, 2846.0, 3630...","[[23200.7, 23451.1, 26337.1, 26587.5, 49610.6,...","[[430577.8, 435351.5, 436091.7, 705504.0, 1110...","[[18107.1, 50904.6, 54633.7, 85868.2, 93871.8,...",[[770]],"[[3296.8], [3812.3], [3824.6], [3836.3], [3960...","[[0.1333073228597641, 0.13327443599700928, 0.1...",...,"[[6904.5, 7401.5, 13545.5, 19718.5, 20719.5, 2...","[[690.35, 740.05, 1354.45, 1971.75, 2071.85, 2...",[[1400]],[[3640]],[[1]],[[934]],[[9]],[[0]],[[[[617.28333333 100.1 784.11666667]]]],"[[690.35, 740.05, 1354.45, 1971.75, 2071.85, 2..."
5,"[[[[[39.60819312]], [[333.01323701]]]]]","[[[[[39.60819312]], [[153.01323701]]]]]",[[[[[ 0.108944]\n [-0.905283]\n [ 0.414634]\n ...,"[[4230.9, 4998.8, 6014.2, 6063.2, 7030.9, 7079...","[[13664.9, 13915.6, 25314.8, 25565.5, 37348.1,...",[],"[[6488.4, 14503.8, 29146.2, 41117.2, 50406.5, ...",[[3840]],"[[5359.6], [5418.3], [5480.0], [5491.0], [5508...","[[0.987357616

In [61]:
print(type(full_data))

# i want to see the middle of the table sometimes. this just drops the stuff in the first half without overriding the data itself.
full_data.drop(labels=["LeftStimulusPosition", "RightStimulusPosition", "EyeWindows", "PhotoEvents", "StimEvents", "Key0", "Key1"], inplace=False, axis=1)

bingus = full_data.assign(LeftStimPos = lambda x: ((x["LeftStimulusPosition"])[0][0][0][0][0][0]))

# (x["LeftStimulusPosition"])[0][0][0][1][0][0]

#df = df.assign(Percentage = lambda x: (x['Total_Marks'] /500 * 100))

print(full_data["LeftStimulusPosition"][0][0][0][0][0][0], full_data["LeftStimulusPosition"][0][0][0][1][0][0])

bingus





<class 'pandas.core.frame.DataFrame'>
32.80834840685461 314.1391540872435


,LeftStimulusPosition,RightStimulusPosition,EyeWindows,PhotoEvents,StimEvents,Key0,Key1,dataStart,spikes,analog,...,Start,Stop,Answer,InterTrialInterval,Condition,ResponseTrial,TrialTiming,spikes,PhotoEventsRHD,LeftStimPos
0,"[[[[[32.80834841]], [[314.13915409]]]]]","[[[[[32.80834841]], [[134.13915409]]]]]",[[[[[-0.214634 ]\n [-0.536585 ]\n [ 0.17561 ]...,"[[1545.7, 1595.5, 2212.8, 2677.1, 2777.1, 3711...","[[176803.3, 177052.9, 182542.4, 182792.0, 1883...","[[91575.8, 258175.6, 386316.3, 388639.5, 53569...","[[12113.9, 17002.0, 21667.3, 27552.4, 56411.8,...",[[770]],"[[828.4], [922.0], [954.3], [984.7], [1040.0],...","[[-0.2835116982460022, -0.2764776349067688, -0...",...,[[1480]],[[3720]],[[1]],[[800]],[[9]],[[0]],[[[[467.13333333 100.1 934.26666667]]]],"[[1499.2], [1525.3], [1534.0], [1543.3], [1551...","[[764.15, 813.85, 1431.15, 1895.45, 1995.55, 2...",32.808348
1,"[[[[[13.46757377]], [[308.20541614]]]]]","[[[[[13.46757377]], [[128.20541614]]]]]",[[[[[ 0.0882702]\n [-1.40488 ]\n [ 0.478514 ]...,"[[10239.8, 10288.6, 10890.4, 11390.9, 11591.1,...","[[15796.2, 16046.5, 24705.2, 24955.5, 27758.2,...","[[289559.0, 466222.6, 467862.5, 470716.4, 1750...","[[11864.1, 19415.9, 31670.0, 37606.1, 96241.3,...",[[9670]],"[[9675.3], [9796.1], [10073.1], [10075.7], [10...","[[0.5390381813049316, 0.5391367673873901, 0.53...",...,[[10180]],[[11390]],[[4]],[[667]],[[0]],[[1]],[[[[500.5 100.1 900.9]]]],"[[10229.0], [10269.4], [10346.3], [10372.7], [...","[[565.25, 614.05, 1215.85]]",32.808348
2,"[[[[[27.21131657]], [[31.50426672]]]]]","[[[[[27.21131657]], [[-148.49573328]]]]]",[[[[[-0.145528]\n [-0.880778]\n [ 0.247967]\n ...,"[[1184.6, 1234.3, 1852.0, 2185.2, 2285.3, 3353...",[],"[[266555.2, 461743.7, 463431.6, 463844.6, 1078...","[[33017.1, 49572.9, 69973.1, 71299.2, 85777.0,...",[[770]],"[[774.3], [1051.6], [1057.8], [1107.5], [1110....","[[-1.309370756149292, -1.310192584991455, -1.3...",...,[[1120]],[[3360]],[[1]],[[850]],[[10]],[[0]],[[[[ 333.66666667 100.1 1067.73333333]]]],"[[1120.9], [1163.6], [1585.4], [2108.3], [2153...","[[410.65, 460.35, 1078.05, 1411.35, 1511.35, 2...",32.808348
3,"[[[[[16.14765063]], [[51.77837553]]]]]","[[[[[16.14765063]], [[-128.22162447]]]]]",[[[[[-0.127235 ]\n [-0.997852 ]\n [ 0.26626 ]...,"[[6017.7, 6066.6, 6701.8, 6750.7, 7369.1, 7969...","[[13208.9, 13459.7, 24970.8, 25221.6, 36982.9,...","[[6466.6, 495471.5, 532818.9, 1073571.9, 13918...","[[14319.7, 29082.0, 40853.1, 49555.5, 58301.3,...",[[4260]],"[[6561.5], [6711.2], [6718.2], [6773.8], [6778...","[[0.07552260905504227, 0.0755554735660553, 0.0...",...,[[5960]],[[6470]],[[3]],[[650]],[[6]],[[0]],[[[[ 367.03333333 100.1 1034.36666667]]]],[],"[[1754.65, 1803.55]]",32.808348
4,"[[[[[41.19192024]], [[334.57146374]]]]]","[[[[[41.19192024]], [[154.57146374]]]]]",[[[[[-0.0491876]\n [-0.965332 ]\n [ 0.268293 ]...,"[[1464.5, 1514.2, 2128.6, 2745.9, 2846.0, 3630...","[[23200.7, 23451.1, 26337.1, 26587.5, 49610.6,...","[[430577.8, 435351.5, 436091.7, 705504.0, 1110...","[[18107.1, 50904.6, 54633.7, 85868.2, 93871.8,...",[[770]],"[[3296.8], [3812.3], [3824.6], [3836.3], [3960...","[[0.1333073228597641, 0.13327443599700928, 0.1...",...,[[1400]],[[3640]],[[1]],[[934]],[[9]],[[0]],[[[[617.28333333 100.1 784.11666667]]]],[[3296.8]],"[[690.35, 740.05, 1354.45, 1971.75, 2071.85, 2...",32.808348
5,"[[[[[39.60819312]], [[333.01323701]]]]]","[[[[[39.60819312]], [[153.01323701]]]]]",[[[[[ 0.108944]\n [-0.905283]\n [ 0.414634]\n ...,"[[4230.9, 4998.8, 6014.2, 6063.2, 7030.9, 7079...","[[13664.9, 13915.6, 25314.8, 25565.5, 37348.1,...",[],"[[6488.4, 14503.8, 29146.2, 41117.2, 50406.5, ...",[[3840]],"[[5359.6], [5418.3], [5480.0], [5491.0], [5508...","[[0.9873576164245605, 0.9870945811271667, 0.98...",...,[[4140]],[[4720]],[[15]],[[733]],[[6]],[[0]],[[[[550 100 850]]]],[],[[382.55]],32.808348
6,"[[[[[18.58280482]], [[272.78354237]]]]]","[[[[[18.58280482]], [[92.78354237]]]]]",[[[[[-0.0536573]\n [-0.933335 ]\n [ 0.252033 ]...,"[[1946.3, 1995.3, 2579.6, 3312.9, 3362.0, 3979..

In [97]:

file = loadmat('hbp-d000034_AttentionNetworksInMonkeys_pub/timestamps/all/20180129_all.mat')
the_important_stuff = file['neurons']

#file
#df_1 = pd.DataFrame(data_1[0][0])

the_stuff = None

#the_important_stuff.shape[0]
for i in range(0, the_important_stuff.shape[0]):
    frame_of_some_sort = pd.DataFrame(the_important_stuff[i][0][0][0][2])
    the_stuff = pd.concat([the_stuff, frame_of_some_sort], axis=1)
#pd.DataFrame(the_important_stuff[0][0])

print(the_important_stuff[0][0][0][0][2])
print("––––––––––––")
print(the_important_stuff[0][0][0][0])
print("––––––––––––")
print(the_important_stuff)

the_stuff





[[1.7240000e-01]
 [2.4870000e-01]
 [3.5360000e-01]
 ...
 [4.2255835e+03]
 [4.2256108e+03]
 [4.2256437e+03]]
––––––––––––
(array(['neural08_A-008a'], dtype='<U15'), array([[500]], dtype=uint16), array([[1.7240000e-01],
       [2.4870000e-01],
       [3.5360000e-01],
       ...,
       [4.2255835e+03],
       [4.2256108e+03],
       [4.2256437e+03]]), array([[1]], dtype=uint8), array([[0]], dtype=uint8))
––––––––––––
[[array([[(array(['neural08_A-008a'], dtype='<U15'), array([[500]], dtype=uint16), array([[1.7240000e-01],
                 [2.4870000e-01],
                 [3.5360000e-01],
                 ...,
                 [4.2255835e+03],
                 [4.2256108e+03],
                 [4.2256437e+03]]), array([[1]], dtype=uint8), array([[0]], dtype=uint8))                 ]],
        dtype=[('name', 'O'), ('varNex5Version', 'O'), ('timestamps', 'O'), ('unitNumber', 'O'), ('wireNumber', 'O')])]
 [array([[(array(['neural10_A-010a'], dtype='<U15'), array([[500]], dtype=uint16), arr

,0,0,0,0,0,0,0,0,0,0,0
0,0.1724,0.5482,0.5563,0.0447,0.7022,0.0986,0.2447,0.0475,0.2669,0.2946,0.5827
1,0.2487,0.5738,0.9636,0.7335,0.9418,0.2423,0.4080,0.1214,0.3843,0.3368,0.5844
2,0.3536,0.5901,1.1927,1.7007,1.2664,0.2740,0.5800,0.1382,0.3917,0.7408,1.6750
3,0.4580,0.8593,1.2371,1.9397,1.6948,0.4807,0.6579,0.1939,0.4185,0.7577,5.2213
4,0.4706,0.9094,1.3109,2.4797,2.3061,0.5612,0.6890,0.3577,0.4454,0.7685,5.2295
...,...,...,...,...,...,...,...,...,...,...,...
68008,NaN,NaN,NaN,NaN,NaN,NaN,4225.2892,NaN,NaN,NaN,NaN
68009,NaN,NaN,NaN,NaN,NaN,NaN,4225.3757,NaN,NaN,NaN,NaN
68010,NaN,NaN,NaN,NaN,NaN,NaN,4225.4595,NaN,NaN,NaN,NaN
68011,NaN,NaN,NaN,NaN,NaN,NaN,4225.4626,NaN,NaN,NaN,NaN


In [154]:

file = loadmat('hbp-d000034_AttentionNetworksInMonkeys_pub/timestamps/SortedByTrial/20180129_SortedByTrial.mat')
the_important_stuff = file['timestamps']

the_important_stuff[0][1483][2][0][1][0][0] == the_important_stuff[0][-2][2][0][1][0][0]

the_important_stuff[0][the_important_stuff[0].size-1]





array([[(array(['neural08_A-008a'], dtype='<U15'), array([], shape=(0, 1), dtype=uint8))],
       [(array(['neural10_A-010a'], dtype='<U15'), array([], shape=(0, 1), dtype=uint8))],
       [(array(['neural10_A-010b'], dtype='<U15'), array([[4223767.35],
               [4223877.85],
               [4223948.85],
               [4223957.75],
               [4224059.25],
               [4224157.05]]))                                         ],
       [(array(['neural13_A-013a'], dtype='<U15'), array([[4222997.05],
               [4223195.05],
               [4223270.35],
               [4223377.35],
               [4223603.95],
               [4223676.45],
               [4223751.55],
               [4223946.75],
               [4224259.95],
               [4224651.25],
               [4224675.45],
               [4224736.45],
               [4224855.35]]))                                         ],
       [(array(['neural13_A-013b'], dtype='<U15'), array([[4224081.65],
               [422

In [118]:
# -------------------------------------------------------------------------------------------------------------
# THIS CELL IS ABOUT FINALLY CLEANING THE DATA NOW THAT I'VE EXPLORED ITS MANY POSSIBLITIES
# Dear god why is it 1:05 AM
# Nevermind it's 2:13 AM
# -------------------------------------------------------------------------------------------------------------

polished_data = full_data


# Focal Point = [leftstim, left+right/2]
focal_point = None

for i in range(0, polished_data.shape[0]):
    pos1 = (full_data["LeftStimulusPosition"][i])[0][0][0][0][0]
    this_point = [pos1, (((full_data["LeftStimulusPosition"][i])[0][0][1][0][0] + (full_data["RightStimulusPosition"][i])[0][0][1][0][0])/2)]
    this_point_in_a_df = pd.DataFrame([[this_point]])
    focal_point = pd.concat([focal_point, this_point_in_a_df], ignore_index=True, axis=0)

polished_data = pd.concat([polished_data, focal_point], axis=1)
polished_data = polished_data.rename(columns={0: 'FocalPoint'})

# oh right. I should probably keep things to one number for analysis. time to do x, y, and distance to 0!

focal_point_y = None
for i in range(0, polished_data.shape[0]):
    pos_y = polished_data.FocalPoint[i][0]
    pos_y_in_a_df = pd.DataFrame([pos_y])
    focal_point_y = pd.concat([focal_point_y, pos_y_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, focal_point_y], axis=1)
polished_data = polished_data.rename(columns={0: 'FocalPointY'})

focal_point_x = None
for i in range(0, polished_data.shape[0]):
    pos_x = polished_data.FocalPoint[i][1]
    pos_x_in_a_df = pd.DataFrame([pos_x])
    focal_point_x = pd.concat([focal_point_x, pos_x_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, focal_point_x], axis=1)
polished_data = polished_data.rename(columns={0: 'FocalPointX'})

focal_dist_to_0 = None
for i in range(0, polished_data.shape[0]):
    foc_0 = math.sqrt(math.pow(polished_data.FocalPointX[i], 2) + math.pow(polished_data.FocalPointY[i], 2))
    foc_0_in_a_df = pd.DataFrame([foc_0])
    focal_dist_to_0 = pd.concat([focal_dist_to_0, foc_0_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, focal_dist_to_0], axis=1)
polished_data = polished_data.rename(columns={0: 'FocalDistTo0'})


# Accuracy Percentage. This can be used as a proxy for how much the monkey was paying attention.

accuracy_percentage = None
for i in range(0, polished_data.shape[0]):
    
    percentage = 0.75  # Some the data didn't include the number of failures. I'm assuming this is some manner of bug with the data collection, and I'm putting any success rates withouth any failures recorded on the lower end of the scale. Arbitrarily.
    if polished_data.Key0[i].size != 0:
        percentage = (polished_data.Key1[i][0].size / (polished_data.Key1[i][0].size + polished_data.Key0[i].size))
    
    percentage_in_a_df = pd.DataFrame([percentage])
    accuracy_percentage = pd.concat([accuracy_percentage, percentage_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, accuracy_percentage], axis=1)
polished_data = polished_data.rename(columns={0: 'AccuracyPercentage'})


# Start and Stop can be used to determine if starting sooner/later affects things

start = None
for i in range(0, polished_data.shape[0]):
    start_time = polished_data.Start[i][0][0]
    start_time_in_a_df = pd.DataFrame([start_time])
    start = pd.concat([start, start_time_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, start], axis=1)
polished_data = polished_data.rename(columns={0: 'StartTime'})

stop = None
for i in range(0, polished_data.shape[0]):
    stop_time = polished_data.Stop[i][0][0]
    stop_time_in_a_df = pd.DataFrame([stop_time])
    stop = pd.concat([stop, stop_time_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, stop], axis=1)
polished_data = polished_data.rename(columns={0: 'StopTime'})


# Also trial length. Stop-start

polished_data = polished_data.assign(TrialLength = lambda x: (x['StopTime'] - x['StartTime']))


# InterTrialInterval

interval = None
for i in range(0, polished_data.shape[0]):
    interval_time = polished_data.InterTrialInterval[i][0][0]
    interval_time_in_a_df = pd.DataFrame([interval_time])
    interval = pd.concat([interval, interval_time_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, interval], axis=1)
polished_data = polished_data.rename(columns={0: 'TrialInterval'})


# try getting number of spikes? neuron activation?

spike = None
for i in range(0, polished_data.shape[0]):
    spike_count = polished_data.spikes[i].size
    spike_count_in_a_df = pd.DataFrame([spike_count])
    spike = pd.concat([spike, spike_count_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, spike], axis=1)
polished_data = polished_data.rename(columns={0: 'spikeCount'})


# I dunno man. Whether there's dimming going on or not? I'm really struggling for ideas here.

dimming = None
for i in range(0, polished_data.shape[0]):
    is_it_dim = polished_data.ResponseTrial[i][0][0] == 1
    is_it_dim_in_a_df = pd.DataFrame([is_it_dim])
    dimming = pd.concat([dimming, is_it_dim_in_a_df], ignore_index=True, axis=0)
polished_data = pd.concat([polished_data, dimming], axis=1)
polished_data = polished_data.rename(columns={0: 'DimConditions'})


polished_data.drop(labels=["LeftStimulusPosition", "RightStimulusPosition", "EyeWindows", "PhotoEvents", "StimEvents", "Key0", "Key1", "dataStart", "spikes", "analog", "digital", "photoEventsRhd", "photoEventsRhdMSec", "Start", "Stop", "Answer", "InterTrialInterval", "Condition", "ResponseTrial", "TrialTiming", "PhotoEventsRHD", "FocalPoint"], inplace=True, axis=1)

polished_data





,FocalPointY,FocalPointX,FocalDistTo0,AccuracyPercentage,StartTime,StopTime,TrialLength,TrialInterval,spikeCount,DimConditions
0,32.808348,224.139154,226.527588,0.955720,1480,3720,2240,800,46013,False
1,13.467574,218.205416,218.620628,0.960699,10180,11390,1210,667,54286,True
2,27.211317,-58.495733,64.515165,0.937063,1120,3360,2240,850,58266,False
3,16.147651,-38.221624,41.492640,0.894081,5960,6470,510,650,78116,False
4,41.191920,244.571464,248.016078,0.938650,1400,3640,2240,934,21511,False
5,39.608193,243.013237,246.219906,0.750000,4140,4720,580,733,46298,False
6,18.582805,182.783542,183.725730,0.750000,1900,3090,1190,833,67788,False
7,23.701323,186.530554,188.030317,0.750000,2450,3200,750,750,50927,False
8,39.832468,235.268251,238.616377,0.750000,22840,25050,2210,616,86330,False
9,43.786907,256.412090,260.123919,0.750000,3690,6470,2780,683,53002,True


In [183]:
# -------------------------------------------------------------------------------------------------------------
# THIS CELL IS ABOUT FINALLY FINALLY FINALLY DOING SOME ACTUAL ANALYSIS
# It is 4:29 AM. That means I have exactly four hours to do this data analysis and make the presentation.
# I think I can do this. I just wish I didn't have to.
# -------------------------------------------------------------------------------------------------------------




# copy the data 
normalized_data = polished_data.copy() 

# gotta turn dimming back into an int for the normalization process
dimming = None
for i in range(0, normalized_data.shape[0]):
    is_it_dim = 0
    if normalized_data['DimConditions'][i]:
        is_it_dim = 1
    is_it_dim_in_a_df = pd.DataFrame([is_it_dim])
    dimming = pd.concat([dimming, is_it_dim_in_a_df], ignore_index=True, axis=0)
normalized_data['DimConditions'] = dimming
  
# apply normalization techniques 
for column in normalized_data.columns: 
    normalized_data[column] = normalized_data[column]  / normalized_data[column].abs().max() 
      

# making analyzed data. this is just polished data, but i'll keep adding columns to it for the clustering stuff.
analyzed_data = polished_data

# adding kmeans cluster to analyzed data.
k = 3
kmeans = KMeans(n_clusters = k)
kmeans.fit(normalized_data)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

analyzed_data = pd.concat([analyzed_data, pd.DataFrame(labels)], axis=1)
analyzed_data = analyzed_data.rename(columns={0: 'K-means Cluster'})
# The centroids didn't tell me much. Worth a shot.
#print("Centroids:", centroids)


# adding knn clustering to analyzed data.
k = 3
nn_cluster = AgglomerativeClustering(n_clusters=k, linkage='ward')
nn_cluster.fit(normalized_data)
labels = nn_cluster.labels_

analyzed_data = pd.concat([analyzed_data, pd.DataFrame(labels)], axis=1)
analyzed_data = analyzed_data.rename(columns={0: 'K-NN Cluster'})


# doing a linear regression with the analyzed data.
linear_model = LinearRegression()
linear_model.fit(X=normalized_data.drop(labels=["AccuracyPercentage"], inplace=False, axis=1), y=normalized_data["AccuracyPercentage"])
coefficients = linear_model.coef_
intercept = linear_model.intercept_

# making the table for all the coefficients.
df_coefficients = pd.DataFrame(normalized_data.columns.values)
df_coefficients = df_coefficients.rename(columns={0: 'Variables'})

index_accuracyPercent = df_coefficients[ df_coefficients['Variables'] == 'AccuracyPercentage'].index
df_coefficients.drop(index_accuracyPercent, inplace = True)
df_coefficients = df_coefficients.reset_index()
del df_coefficients['index']
df_coefficients = pd.concat([df_coefficients, pd.DataFrame(coefficients)], axis=1)
df_coefficients = df_coefficients.rename(columns={0: 'Coefficients'})


# Why not, let's get some one-way ANOVA in there so I can see just what's statiscally significant here!
f_stat, p_value = f_oneway(normalized_data['FocalPointY'], normalized_data['FocalPointX'], normalized_data['FocalDistTo0'], normalized_data['StartTime'], normalized_data['StopTime'], normalized_data['TrialLength'], normalized_data['TrialInterval'], normalized_data['spikeCount'], normalized_data['DimConditions'])


# Actually plotting the line.

b = intercept
m = 
# I can change x around!

regression_line = m * x + b

x = df_coefficients
y = polished_data['AccuracyPercentage']

# Plotting a line plot
plt.plot(x, y, color='skyblue', linewidth=2)
plt.title('Linear Regression')
plt.xlabel('Coefficients')
plt.ylabel('Accuracy Percentage')
plt.show()



print("F-statistic:", f_stat, "\nP-value:", p_value)
display(analyzed_data.sort_values(by="K-means Cluster"))
display(df_coefficients)
print("Intercept:", intercept)




F-statistic: 17.679743884271545 
P-value: 1.4447885638853998e-19


,FocalPointY,FocalPointX,FocalDistTo0,AccuracyPercentage,StartTime,StopTime,TrialLength,TrialInterval,spikeCount,DimConditions,K-means Cluster,K-NN Cluster
10,45.635113,248.164269,252.325322,0.978610,1290,2940,1650,833,47812,True,0,2
1,13.467574,218.205416,218.620628,0.960699,10180,11390,1210,667,54286,True,0,2
9,43.786907,256.412090,260.123919,0.750000,3690,6470,2780,683,53002,True,0,2
19,20.000000,240.000000,240.831892,0.750000,6720,6840,120,900,87430,False,1,0
16,14.809997,212.029466,212.546066,0.750000,8610,10840,2230,950,79544,False,1,0
15,38.709826,258.147419,261.033600,0.750000,1050,3230,2180,816,60496,False,1,0
14,32.739582,223.389560,225.775941,0.750000,2880,3010,130,916,91255,False,1,0
13,40.317126,240.307703,243.666294,0.750000,14400,16610,2210,883,25391,False,1,0
12,41.107577,235.581951,239.141566,0.780347,980,3170,2190,750,12817,False,1,0
11,24.577486,183.955470,185.590053,0.961832,28450,28530,80,833,125811,False,1,0


,Variables,Coefficients
0,FocalPointY,0.111777
1,FocalPointX,0.322305
2,FocalDistTo0,-0.706970
3,StartTime,0.059279
4,StopTime,0.052678
5,TrialLength,-0.066036
6,TrialInterval,0.250926
7,spikeCount,-0.159276
8,DimConditions,0.152593


Intercept: 0.933202756996121
